In [9]:
## Use scikit-learn to grid search the batch size and epochs
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(n_layers=1, optimizer='adam', init_mode='uniform'):
    '''Function to create model, required for KerasClassifier'''
    ## create model
    model = Sequential()
    for i in range(0, n_layers):
        model.add(Dense(12, input_dim=8, kernel_initializer=init_mode activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    ## Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

## fix random seed for reproducibility
seed = 7
np.random.seed(seed)
## load dataset
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
## split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
## create model
model = KerasClassifier(build_fn=create_model, verbose=0)
## define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
# batch_size = list(np.arange(5, 200, 5))
epochs = [10, 50, 100]
# epochs = list(np.arange(1, 2000, 2))
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)
from datetime import datetime
start=datetime.now()
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
print(datetime.now()-start)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0:09:41.161886
Best: 0.688802 using {'epochs': 100, 'batch_size': 80}
0.429688 (0.138254) with: {'epochs': 10, 'batch_size': 10}
0.559896 (0.115453) with: {'epochs': 50, 'batch_size': 10}
0.679688 (0.019918) with: {'epochs': 100, 'batch_size': 10}
0.466146 (0.137566) with: {'epochs': 10, 'batch_size': 20}
0.665365 (0.014731) with: {'epochs': 50, 'batch_size': 20}
0.669271 (0.031948) with: {'epochs': 100, 'batch_size': 20}
0.604167 (0.062201) with: {'epochs': 10, 'batch_size': 40}
0.667969 (0.044993) with: {'epochs': 50, 'batch_size': 40}
0.635417 (0.020752) with: {'epochs': 100, 'batch_size': 40}
0.618490 (0.045814) with: {'epochs': 10, 'batch_size': 60}
0.643229 (0.032734) with: {'epochs': 50, 'batch_size': 60}
0.667969 (0.054408) with: {'epochs': 100, 'batch_size': 60}
0.557292 (0.018136) with: {'epochs': 10, 'batch_size': 80}
0.674479 (0.046256) with: {'epochs': 50, 'batch_size': 80}
0.688802 (0.021236) with: {'epochs': 100, 'batch_size': 80}
0.546875 (0.048159) with: {'epochs': 10,